In [1]:
from __future__ import annotations

import heapq
import math
import os
import re
import sys
import time
from collections import defaultdict, deque
from itertools import combinations
from typing import Dict, List, Tuple

import aocd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib.path import Path
from ortools.sat.python import cp_model
from scipy.ndimage import convolve
from shapely.geometry import Polygon, box
from tqdm.notebook import tqdm, trange
from z3 import Int, Optimize, sat

In [2]:
p = aocd.get_puzzle(year=2025, day=12)

In [3]:
p.examples[0].input_data

'0:\n###\n##.\n##.\n\n1:\n###\n##.\n.##\n\n2:\n.##\n###\n##.\n\n3:\n##.\n###\n##.\n\n4:\n###\n#..\n###\n\n5:\n###\n.#.\n###\n\n4x4: 0 0 0 0 2 0\n12x5: 1 0 1 0 2 2\n12x5: 1 0 1 0 3 2'

In [4]:
def get_data(test_data: bool = True):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data
    return data

In [5]:
def process_data(data):
    _shapes = data.split("\n\n")[:-1]
    _grids = data.split("\n\n")[-1]
    shapes = {}
    grids = []
    for s in _shapes:
        i = int(s.split(":")[0])
        _shape = s.split(":")[1].split("\n")[1:]
        _shape = np.array([[1 if c == "#" else 0 for c in row] for row in _shape])
        shapes[i] = _shape
    for s in _grids.split("\n"):
        i = tuple([int(match) for match in re.findall(r"\d+", s.split()[0])])
        r = [int(t) for t in s.split()[1:]]
        g = (i, r)
        grids.append(g)

    return shapes, grids

In [6]:
data = get_data(test_data=False)
pieces, grids = process_data(data)

In [7]:
# Even more simple

In [8]:
sum(1 for size, counts in grids if (size[0] * size[1] // 9) >= sum(counts))

565